In [1]:
# ============================================================
#  Neural Network Surrogate for Black-Box Optimisation (BBO)
# ============================================================
#  Overview:
#  This code trains a PyTorch neural network surrogate model to
#  approximate a black-box function and identify the next best
#  query point via Monte Carlo sampling.
#
#  Core idea:
#  - The surrogate learns the relationship between inputs (X)
#    and outputs (y).
#  - Monte Carlo sampling generates many random candidates.
#  - The surrogate predicts their outputs.
#  - The best candidate (highest predicted y) is chosen as the
#    next query point.
#
#  Monte Carlo approach:
#  ---------------------
#  Since black-box functions are unknown and costly to query,
#  Monte Carlo sampling provides an efficient, probabilistic
#  method to explore the input space and estimate promising
#  regions for improvement.
#
#  Exploration vs. Exploitation Controls:
#  --------------------------------------
#  - Learning Rate (lr):
#       Higher = faster, noisier updates (more exploration)
#       Lower  = slower, stable convergence (more exploitation)
#  - Batch Size (batch_size):
#       Smaller = more stochastic gradient (exploration)
#       Larger  = smoother gradient, focus on known good regions (exploitation)
#  - Monte Carlo Samples (n_candidates):
#       More = broader coverage of search space (exploration)
#       Fewer = faster computation, focus on known regions (exploitation)
#
# ============================================================
#
# v0.02 - Full rewrite using Pytorch with clear functions for easier understanding.
# v0.03 - Updated to use GP to explore and shortlist a region, then use NN for
#         exploitation within the shortlisted region.
# v0.04 - Updated to down-weight low importance dimensions
# v0.05 - Exploitation started. Updated a new generate sample candidates function
#         to generate importance weighted candidates in the exploitation region.
#         Also updated gp_acq_k to exploit
# v0.06 - Exploitation continuing with tighter hyperparameters:
#         stage1_size = 12000
#         stage2_size = 1000
#         gp_acq_k    = 0.3
#         nn_batch    = 512
#         base_radius = 0.15
# v0.07 - Exploitation continuing with tighter hyperparameters:
#         stage1_size   = 10000
#         stage2_size   = 800
#         gp_acq_k      = 0.20
#         nn_batch      = 512
#         base_radius   = 0.12


In [2]:
# Mount drive for loading initial input and output files
def mount_drive():
  from google.colab import drive
  drive.mount('/content/drive')

mount_drive()

# Import Libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel as C

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

Mounted at /content/drive


In [3]:
# Load the initial files
def load_initial_files(function='1'):
  path = 'drive/MyDrive/data/'
  given_X = path + 'fn' + function + '_initial_inputs.npy'
  given_y = path + 'fn' + function + '_initial_outputs.npy'
  X = np.load(given_X)        # shape (n0, d)
  y = np.load(given_y)       # shape (n0,)

  return X, y

# Append new data from recent queries
def append_new_data(X, y, function):
  if function == '1':
    X = np.append(X,[[0.701438, 0.897524]], axis=0)  # Append week1 inputs
    y = np.append(y, 4.907008577514035e-55)         # Append week1 outputs
    X = np.append(X,[[0.608788, 0.208839]], axis=0)  # Append week2 inputs
    y = np.append(y, -3.407789887257921e-57)         # Append week2 outputs
    X = np.append(X,[[0.937387, 0.537998]], axis=0)  # Append week3 inputs
    y = np.append(y, -1.8248983945966781e-72)         # Append week3 outputs
    X = np.append(X,[[0.401144, 0.347316]], axis=0)  # Append week4 inputs
    y = np.append(y, -0.000052943333535254514)         # Append week4 outputs
    X = np.append(X,[[0.815411, 0.802906]], axis=0)  # Append week5 inputs
    y = np.append(y, 2.3120136577868056e-46)         # Append week5 outputs
    X = np.append(X,[[0.147927, 0.999351]], axis=0)  # Append week6 inputs
    y = np.append(y, -1.0556884679861184e-256)         # Append week6 outputs
    X = np.append(X,[[0.961902, 0.541510]], axis=0)  # Append week7 inputs
    y = np.append(y, -4.446243588517271e-83)         # Append week7 outputs
    X = np.append(X,[[0.354466, 0.463725]], axis=0)  # Append week8 inputs
    y = np.append(y, 0.000004670359491308217)         # Append week8 outputs
    X = np.append(X,[[0.200840, 0.740525]], axis=0)  # Append week9 inputs
    y = np.append(y, -2.223577503605569e-105)         # Append week9 outputs
    X = np.append(X,[[0.329036, 0.431477]], axis=0)  # Append week10 inputs
    y = np.append(y, -7.726124369273896e-7)         # Append week10 outputs
  elif function == '2':
    X = np.append(X,[[0.695813, 0.000000]], axis=0)  # Append week inputs
    y = np.append(y, 0.6257440183692108)         # Append week1 outputs
    X = np.append(X,[[0.969061, 0.774664]], axis=0)  # Append week2 inputs
    y = np.append(y, 0.033917749094575816)         # Append week2 outputs
    X = np.append(X,[[0.690739, 0.962975]], axis=0)  # Append week3 inputs
    y = np.append(y, 0.6440194651539941)         # Append week3 outputs
    X = np.append(X,[[0.000000, 0.982469]], axis=0)  # Append week4 inputs
    y = np.append(y, 0.011920662637457252)         # Append week4 outputs
    X = np.append(X,[[0.819992, 0.888300]], axis=0)  # Append week5 inputs
    y = np.append(y, -0.05572035846713359)         # Append week5 outputs
    X = np.append(X,[[0.576225, 0.995582]], axis=0)  # Append week6 inputs
    y = np.append(y, -0.05917879343392843)         # Append week6 outputs
    X = np.append(X,[[0.686326, 0.048467]], axis=0)  # Append week7 inputs
    y = np.append(y, 0.5502098056853213)         # Append week7 outputs
    X = np.append(X,[[0.684793, 0.736977]], axis=0)  # Append week8 inputs
    y = np.append(y, 0.6051181573714066)         # Append week8 outputs
    X = np.append(X,[[0.697663, 0.999431]], axis=0)  # Append week9 inputs
    y = np.append(y, 0.6074029564402459)         # Append week9 outputs
    X = np.append(X,[[0.684762, 0.956283]], axis=0)  # Append week10 inputs
    y = np.append(y, 0.5911243348817143)         # Append week10 outputs
  elif function == '3':
    X = np.append(X,[[0.000000, 0.000000, 0.000000]], axis=0)  # Append week1 inputs
    y = np.append(y, -0.1755204613669372)         # Append week1 outputs
    X = np.append(X,[[0.999999, 0.183024, 0.788109]], axis=0)  # Append week2 inputs
    y = np.append(y, -0.10594389054163186)         # Append week2 outputs
    X = np.append(X,[[0.937840, 0.999999, 0.405928]], axis=0)  # Append week3 inputs
    y = np.append(y, -0.05889151139553504)         # Append week3 outputs
    X = np.append(X,[[0.523777, 0.465195, 0.442681]], axis=0)  # Append week4 inputs
    y = np.append(y, -0.005996811424753983)         # Append week4 outputs
    X = np.append(X,[[0.503614, 0.496940, 0.415922]], axis=0)  # Append week5 inputs
    y = np.append(y, -0.0062783618309617175)         # Append week5 outputs
    X = np.append(X,[[0.306294, 0.000901, 0.737840]], axis=0)  # Append week6 inputs
    y = np.append(y, -0.1868873865434488)         # Append week6 outputs
    X = np.append(X,[[0.785819, 0.951069, 0.007398]], axis=0)  # Append week7 inputs
    y = np.append(y, -0.12354269078229708)         # Append week7 outputs
    X = np.append(X,[[0.671012, 0.540880, 0.410996]], axis=0)  # Append week8 inputs
    y = np.append(y, -0.016078870026236092)         # Append week8 outputs
    X = np.append(X,[[0.511600, 0.466301, 0.440801]], axis=0)  # Append week9 inputs
    y = np.append(y, -0.003448118792089531)         # Append week9 outputs
    X = np.append(X,[[0.541094, 0.476141, 0.429833]], axis=0)  # Append week10 inputs
    y = np.append(y, -0.006776539824628652)         # Append week10 outputs
  elif function == '4':
    X = np.append(X,[[0.440415, 0.425453, 0.378353, 0.397106]], axis=0)  # Append week1 inputs
    y = np.append(y, 0.2601025838576061)         # Append week1 outputs
    X = np.append(X,[[0.410011, 0.415255, 0.344454, 0.438808]], axis=0)  # Append week2 inputs
    y = np.append(y, 0.3983591561199096)         # Append week2 outputs
    X = np.append(X,[[0.384038, 0.412994, 0.399499, 0.436890]], axis=0)  # Append week3 inputs
    y = np.append(y, 0.2987152140170406)         # Append week3 outputs
    X = np.append(X,[[0.391529, 0.451226, 0.356835, 0.420242]], axis=0)  # Append week4 inputs
    y = np.append(y, 0.06384840143364956)         # Append week4 outputs
    X = np.append(X,[[0.529644, 0.477474, 0.456444, 0.497979]], axis=0)  # Append week5 inputs
    y = np.append(y, -3.620742573150156)         # Append week5 outputs
    X = np.append(X,[[0.362616, 0.390935, 0.358916, 0.408803]], axis=0)  # Append week6 inputs
    y = np.append(y, 0.5941496830193782)         # Append week6 outputs
    X = np.append(X,[[0.408168, 0.424670, 0.350333, 0.464759]], axis=0)  # Append week7 inputs
    y = np.append(y, -0.28307671964285275)         # Append week7 outputs
    X = np.append(X,[[0.462778, 0.459808, 0.427924, 0.383695]], axis=0)  # Append week8 inputs
    y = np.append(y, -1.072387515133428)         # Append week8 outputs
    X = np.append(X,[[0.337602, 0.314128, 0.323601, 0.447251]], axis=0)  # Append week9 inputs
    y = np.append(y, -0.9951880222627056)         # Append week9 outputs
    X = np.append(X,[[0.379054, 0.415621, 0.376105, 0.405300]], axis=0)  # Append week10 inputs
    y = np.append(y, 0.4527725055900622)         # Append week10 outputs
  elif function == '5':
    X = np.append(X,[[0.000000, 0.827185, 0.999999, 0.999999]], axis=0)  # Append week1 inputs
    y = np.append(y, 2781.638812419282)         # Append week1 outputs
    X = np.append(X,[[0.058940, 0.195873, 0.872581, 0.998561]], axis=0)  # Append week2 inputs
    y = np.append(y, 848.8896402098709)         # Append week2 outputs
    X = np.append(X,[[0.166206, 0.999999, 0.999999, 0.999999]], axis=0)  # Append week3 inputs
    y = np.append(y, 4444.256290210307)         # Append week3 outputs
    X = np.append(X,[[0.259704, 0.866240, 0.999999, 0.078304]], axis=0)  # Append week4 inputs
    y = np.append(y, 837.6996715082641)         # Append week4 outputs
    X = np.append(X,[[0.405221, 0.818629, 0.840579, 0.874304]], axis=0)  # Append week5 inputs
    y = np.append(y, 848.6842411794586)         # Append week5 outputs
    X = np.append(X,[[0.379668, 0.980190, 0.997527, 0.939278]], axis=0)  # Append week6 inputs
    y = np.append(y, 3506.2636242750236)         # Append week6 outputs
    X = np.append(X,[[0.165658, 0.993001, 0.985802, 0.828885]], axis=0)  # Append week7 inputs
    y = np.append(y, 2570.8680824924277)         # Append week7 outputs
    X = np.append(X,[[0.450158, 0.310399, 0.558612, 0.719998]], axis=0)  # Append week8 inputs
    y = np.append(y, 1.9175678320859924)         # Append week8 outputs
    X = np.append(X,[[0.106033, 0.999999, 0.999999, 0.999999]], axis=0)  # Append week9 inputs
    y = np.append(y, 4441.287683454212)         # Append week9 outputs
    X = np.append(X,[[0.161038, 0.999999, 0.999999, 0.999999]], axis=0)  # Append week10 inputs
    y = np.append(y, 4443.855101752938)         # Append week10 outputs
  elif function == '6':
    X = np.append(X,[[0.464910, 0.242338, 0.574752, 0.999999, 0.000000]], axis=0)  # Append week1 inputs
    y = np.append(y, -0.5265043497038704)         # Append week1 outputs
    X = np.append(X,[[0.537065, 0.317653, 0.626041, 0.898937, 0.148218]], axis=0)  # Append week2 inputs
    y = np.append(y, -0.41510731628352715)         # Append week2 outputs
    X = np.append(X,[[0.455585, 0.174069, 0.999999, 0.999999, 0.292903]], axis=0)  # Append week3 inputs
    y = np.append(y, -1.0104998624615082)         # Append week3 outputs
    X = np.append(X,[[0.525055, 0.369255, 0.490228, 0.765387, 0.025091]], axis=0)  # Append week4 inputs
    y = np.append(y, -0.3481623700173726)         # Append week4 outputs
    X = np.append(X,[[0.541275, 0.517331, 0.640338, 0.739537, 0.379138]], axis=0)  # Append week5 inputs
    y = np.append(y, -0.6094732138365111)         # Append week5 outputs
    X = np.append(X,[[0.377302, 0.427069, 0.554688, 0.835249, 0.037887]], axis=0)  # Append week6 inputs
    y = np.append(y, -0.3821980197564756)         # Append week6 outputs
    X = np.append(X,[[0.399356, 0.472775, 0.272075, 0.930991, 0.004327]], axis=0)  # Append week7 inputs
    y = np.append(y, -0.7153807487746384)         # Append week7 outputs
    X = np.append(X,[[0.352618, 0.580837, 0.570192, 0.621771, 0.372965]], axis=0)  # Append week8 inputs
    y = np.append(y, -0.7718310362808554)         # Append week8 outputs
    X = np.append(X,[[0.463141, 0.457153, 0.632226, 0.697789, 0.000000]], axis=0)  # Append week9 inputs
    y = np.append(y, -0.3593032465991644)         # Append week9 outputs
    X = np.append(X,[[0.485318, 0.354410, 0.532001, 0.735756, 0.002593]], axis=0)  # Append week10 inputs
    y = np.append(y, -0.2563938551700072)         # Append week10 outputs
  elif function == '7':
    X = np.append(X,[[0.000000, 0.247036, 0.408965, 0.217149, 0.377534, 0.746590]], axis=0)  # Append week1 inputs
    y = np.append(y, 2.3034568430941222)         # Append week1 outputs
    X = np.append(X,[[0.000000, 0.181841, 0.435826, 0.062982, 0.361647, 0.858489]], axis=0)  # Append week2 inputs
    y = np.append(y, 1.3279380639726712)         # Append week2 outputs
    X = np.append(X,[[0.000000, 0.166738, 0.211207, 0.080984, 0.381370, 0.746608]], axis=0)  # Append week3 inputs
    y = np.append(y, 1.433484749556216)         # Append week3 outputs
    X = np.append(X,[[0.000000, 0.198493, 0.750365, 0.247059, 0.370826, 0.980737]], axis=0)  # Append week4 inputs
    y = np.append(y, 1.239424289285777)         # Append week4 outputs
    X = np.append(X,[[0.456843, 0.374177, 0.452545, 0.525118, 0.456228, 0.722847]], axis=0)  # Append week5 inputs
    y = np.append(y, 0.8787401362260749)         # Append week5 outputs
    X = np.append(X,[[0.002100, 0.111359, 0.320991, 0.376242, 0.229457, 0.955090]], axis=0)  # Append week6 inputs
    y = np.append(y, 1.1302577537063068)         # Append week6 outputs
    X = np.append(X,[[0.051607, 0.242133, 0.329607, 0.131528, 0.404377, 0.688463]], axis=0)  # Append week7 inputs
    y = np.append(y, 1.974692744635984)         # Append week7 outputs
    X = np.append(X,[[0.391166, 0.399851, 0.480060, 0.389445, 0.406096, 0.616639]], axis=0)  # Append week8 inputs
    y = np.append(y, 1.794628814733577)         # Append week8 outputs
    X = np.append(X,[[0.000000, 0.273026, 0.493981, 0.248861, 0.418811, 0.645849]], axis=0)  # Append week9 inputs
    y = np.append(y, 2.2957847800972426)         # Append week9 outputs
    X = np.append(X,[[0.000000, 0.246499, 0.426534, 0.258151, 0.365936, 0.706056]], axis=0)  # Append week10 inputs
    y = np.append(y, 2.550792309384511)         # Append week10 outputs
  elif function == '8':
    X = np.append(X,[[0.060275, 0.000000, 0.134973, 0.000000, 0.999999, 0.404343, 0.057755, 0.516689]], axis=0)  # Append week1 inputs
    y = np.append(y, 9.8814582425914)         # Append week1 outputs
    X = np.append(X,[[0.122654, 0.153991, 0.162413, 0.045600, 0.999999, 0.536650, 0.260832, 0.932950]], axis=0)  # Append week2 inputs
    y = np.append(y, 9.9450768395815)         # Append week2 outputs
    X = np.append(X,[[0.085442, 0.318585, 0.000000, 0.239064, 0.999999, 0.927570, 0.142651, 0.932950]], axis=0)  # Append week3 inputs
    y = np.append(y, 9.6920435401985)         # Append week3 outputs
    X = np.append(X,[[0.266427, 0.000000, 0.185600, 0.000000, 0.999999, 0.210401, 0.179882, 0.156373]], axis=0)  # Append week4 inputs
    y = np.append(y, 9.7659726871796)         # Append week4 outputs
    X = np.append(X,[[0.514029, 0.459555, 0.534367, 0.406509, 0.744510, 0.663310, 0.578111, 0.756422]], axis=0)  # Append week5 inputs
    y = np.append(y, 8.6884084301446)         # Append week5 outputs
    X = np.append(X,[[0.026980, 0.613170, 0.025877, 0.359845, 0.896836, 0.008212, 0.028519, 0.131032]], axis=0)  # Append week6 inputs
    y = np.append(y, 9.3709013812716)         # Append week6 outputs
    X = np.append(X,[[0.065492, 0.074877, 0.151838, 0.036631, 0.882332, 0.837480, 0.404222, 0.901998]], axis=0)  # Append week7 inputs
    y = np.append(y, 9.7678761465696)         # Append week7 outputs
    X = np.append(X,[[0.416048, 0.599220, 0.417048, 0.424239, 0.551884, 0.585992, 0.386618, 0.514817]], axis=0)  # Append week8 inputs
    y = np.append(y, 9.1674784539701)         # Append week8 outputs
    X = np.append(X,[[0.032703, 0.075576, 0.154423, 0.035445, 0.990590, 0.497687, 0.174691, 0.986528]], axis=0)  # Append week9 inputs
    y = np.append(y, 9.9361018945346)         # Append week9 outputs
    X = np.append(X,[[0.128787, 0.124725, 0.195336, 0.024228, 0.999999, 0.534909, 0.223820, 0.947785]], axis=0)  # Append week10 inputs
    y = np.append(y, 9.934630153261)         # Append week10 outputs


  return X, y

# Normalise X and y
def normalise_data(X, y):
  x_scaler = StandardScaler()
  Xn = x_scaler.fit_transform(X)
  y_mean = y.mean()
  y_std = y.std() if y.std() > 0 else 1.0
  yn = (y - y_mean) / y_std   # Normalize target manually (avoid sklearn overhead)

  return Xn, yn, y_mean, y_std, x_scaler

In [4]:
# Define Neural Network surrogate model
class SurrogateNN(nn.Module):
    def __init__(self, input_dim, dropout_rate=0.2):
        super(SurrogateNN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

# Train Surrogate NN model with optional mini batching
def train_nn(X_train, y_train, input_dim, lr=0.01, epochs=300, batch_size=None):
    """
    Trains the neural network surrogate.
    If batch_size is None, uses full-batch training.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    nn_model = SurrogateNN(input_dim).to(device)
    optimizer = optim.Adam(nn_model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()

    X_t = torch.tensor(X_train, dtype=torch.float32).to(device)
    y_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1).to(device)

    if batch_size is not None:
        dataset = TensorDataset(X_t, y_t)
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    else:
        loader = [(X_t, y_t)]  # full-batch fallback

    for epoch in range(epochs):
        for xb, yb in loader:
            optimizer.zero_grad()
            preds = nn_model(xb)
            loss = loss_fn(preds, yb)
            loss.backward()
            optimizer.step()

    return nn_model

# Monte Carlo Sampling to Find Next Query Point
def suggest_next_query(nn_model, input_dim, x_scaler, y_mean, y_std, n_candidates=10000):
    """
    Uses Monte Carlo sampling to suggest the next query point.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    nn_model.eval()

    # Generate random candidate points uniformly in [0,1]^d
    candidates = np.random.rand(n_candidates, input_dim)

    #Normalise candidates using fitted scaler
    candidates_norm = x_scaler.transform(candidates)

    # Predict for normalised candidates
    with torch.no_grad():
        preds_norm = model(torch.tensor(candidates_norm, dtype=torch.float32).to(device)).cpu().numpy()

    # Select candidate with highest predicted value
    best_idx = np.argmax(preds_norm)
    x_next_norm = candidates_norm[best_idx]
    y_pred_norm = preds_norm[best_idx]

    # Denormalise back to original scale
    x_next = x_scaler.inverse_transform(x_next_norm.reshape(1, -1)).flatten()
    y_pred = y_pred_norm * y_std + y_mean

    return x_next, y_pred

In [5]:
# Train a surrogate GP model
def train_gp(Xn, yn, kernel=None, n_restarts_optimizer=10, random_state=42):
  if kernel is None:
    d = Xn.shape[1]
    kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=np.ones(d),
                                          length_scale_bounds=(1e-6, 1e3),
                                          nu=2.5)
    kernel += WhiteKernel(noise_level=1e-6, noise_level_bounds=(1e-8, 1e1))

  gp_model = GaussianProcessRegressor(kernel=kernel,
                                      normalize_y=False,
                                      n_restarts_optimizer=n_restarts_optimizer,
                                      random_state=random_state)
  gp_model.fit(Xn, yn)

  return gp_model

# Use NN to predict the shortlisted batch (returns normalised preds)
def nn_predict(nn_model, X_scaled, batch=1024):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  nn_model.to(device)
  nn_model.eval()
  nn_preds = []
  with torch.no_grad():
    for i in range(0, X_scaled.shape[0], batch):
      xb = X_scaled[i:i+batch]
      xb_t = torch.tensor(xb, dtype=torch.float32).to(device)
      preds = nn_model(xb_t).cpu().numpy().reshape(-1)
      nn_preds.append(preds)

  nn_preds = np.concatenate(nn_preds, axis=0)
  return nn_preds

# Two stage approach using GP and NN to find Next Query Point
def suggest_next_query_two_stage(nn_model, gp_model, input_dim, x_scaler, y_mean, y_std,
                                 stage1_size=20000, stage2_size=1000, gp_acq_k=1.96, nn_batch=1024, current_best_x=None, base_radius=0.5):
  """
    Two-stage selection:
      1) Sample stage1_size candidates uniformly in [0,1]^d.
      2) Scale them and evaluate GP (mu, std) -> acquisition = mu + gp_acq_k * std.
      3) Shortlist top `shortlist` candidates by GP acquisition.
      4) Re-rank shortlist by NN predicted mean (NN trained on scaled inputs).
      5) Return raw chosen x (in [0,1]^d) and denormalised predicted y.

    Returns:
      x_next_raw: (d,) raw candidate in [0,1]
      y_pred_raw: float predicted output in original scale
  """
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  # Generate random candidate points uniformly in [0,1]^d
  #candidates = np.random.rand(stage1_size, input_dim)

  # Generate random canditates based on dimension importance
  gp_imp = get_gp_importance(gp_model)
  nn_imp = get_nn_importance(nn_model, Xn)
  importance = combine_importance(gp_imp, nn_imp)
  #candidates = sample_candidates_weighted(importance, n_samples=stage1_size)  # during exploration
  candidates = sample_candidates_weighted_local(importance, n_samples=stage1_size, center=current_best_x, base_radius=base_radius)  # during exploitation


  #Normalise candidates using fitted scaler
  candidates_norm = x_scaler.transform(candidates)

  # GP predict on scaled candidates (returns normalized outputs)
  mu_gp, sigma_gp = gp_model.predict(candidates_norm, return_std=True)     # both shape (stage1_size,)
  # Acquisition: simple UCB-like
  gp_acq = mu_gp + gp_acq_k * sigma_gp

  # Shortlist top-k candidates by GP acquisition
  top_idx = np.argsort(gp_acq)[-stage2_size:]
  X_short_raw = candidates[top_idx]         # raw coordinates (shortlist)
  X_short_scaled = candidates_norm[top_idx]   # scaled coordinates for NN

  nn_preds_norm = nn_predict(nn_model, X_short_scaled, batch=nn_batch)
  # nn_preds_norm are in normalized y-space (because NN trained on yn).
  # We'll rank by normalized preds (same order as raw).
  best_local = np.argmax(nn_preds_norm)
  x_next = X_short_raw[best_local]            # keep in [0,1] space

  # The NN predicted normalized y so we denormalize it
  y_pred_norm = nn_preds_norm[best_local]
  y_pred = float(y_pred_norm * y_std + y_mean)

  return x_next, y_pred



In [6]:
# Approach to down-weight low importance dimensions
# Get nn importance
def get_nn_importance(nn_model, X_scaled):
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  nn_model.to(device)
  nn_model.eval()
  X_t = torch.tensor(X_scaled, dtype=torch.float32, requires_grad=True)
  preds = nn_model(X_t).sum()
  preds.backward()
  grads = X_t.grad.abs().mean(dim=0).numpy()
  return grads / grads.sum()

# Get gp importance
def get_gp_importance(gp_model):
  ls = np.array(gp_model.kernel_.k1.k2.length_scale).flatten()
  raw = 1.0 / (ls + 1e-8)
  return raw / raw.sum()

# Commbine nn_importance and gp_importance
def combine_importance(gp_imp, nn_imp, gp_weight=0.6):
  imp = gp_weight * gp_imp + (1 - gp_weight) * nn_imp
  return imp / imp.sum()

# Generate candidates based on combined importance
def sample_candidates_weighted(importance, n_samples=10000):
    """
    Generate candidate samples where each dimension is sampled
    more narrowly if its importance is low.

    importance: array of shape [d] summing to 1
    """
    dim = len(importance)

    # Scale ranges: important dims sampled wide, weak dims sampled tight.
    # Values stay within [0,1].
    #
    # scale = 1.0 => explore full [0,1]
    # scale = 0.2 => explore only central 20%
    #
    scale = 0.1 + 0.9 * importance  # ensures minimum variety

    # Sample raw uniform values
    raw = np.random.rand(n_samples, dim)

    # Apply scaling around 0.5 (centralised exploration in weak dims)
    candidates = 0.5 + (raw - 0.5) * scale

    # Clip to ensure valid bounds
    candidates = np.clip(candidates, 0.0, 1.0)

    return candidates

# Generate candidates for exploitation based on combined importance
def sample_candidates_weighted_local(importance, n_samples,  center, base_radius=0.1):
    """
    Importance-weighted local sampling around a known good point.

    importance : (d,) importance weights, sum to 1
    center     : (d,) current_best_x in [0,1]
    base_radius: max local radius for most important dimensions
    """
    dim = len(importance)

    # More important dims get larger local radius
    radius = base_radius * (0.1 + 0.9 * importance)

    noise = (np.random.rand(n_samples, dim) - 0.5) * 2.0
    candidates = center + noise * radius

    # Clip to ensure valid bounds
    candidates = np.clip(candidates, 0.0, 1.0)

    return candidates

In [14]:
# Decide function and batch_size
func = '8'       # number between 1 and 9 representing BBO function
batch_size = 8   # number or None

# Load data and normalise
X, y = load_initial_files(function=func)
X, y = append_new_data(X, y, function=func)
current_best_x = X[np.argmax(y)]  # observed best only
Xn, yn, y_mean, y_std, x_scaler = normalise_data(X, y)

print(len(Xn))
print(max(y))
print(max(yn))

# Calcualte input shape
input_dim = Xn.shape[1]

# Train NN model
nn_model = train_nn(Xn, yn, input_dim, lr=0.01, epochs=500, batch_size=batch_size)

# Train GP model
gp_model = train_gp(Xn, yn, kernel=None, n_restarts_optimizer=10, random_state=42)

# Get the Next Query Point
#x_next, y_pred = suggest_next_query(model, input_dim, x_scaler, y_mean, y_std, n_candidates=50000)
# gp_acq_k changed to 0.3 from 1.96 at the start of exploitation

x_next, y_pred = suggest_next_query_two_stage(nn_model, gp_model, input_dim, x_scaler, y_mean, y_std,
  stage1_size=10000, stage2_size=800, gp_acq_k=0.2, nn_batch=512, current_best_x=current_best_x, base_radius=0.12
)

print("\nFunction:", func)
print("\nSuggested next input (x_next):", '-'.join(map(str, np.round(x_next, 6))))
#print("\nPredicted output estimate:", float(y_pred[0]))
print("\nPredicted output estimate:", float(y_pred))


50
9.9450768395815
1.572567695294003

Function: 8

Suggested next input (x_next): 0.090796-0.13846-0.178644-0.020794-0.996394-0.533622-0.231853-0.939059

Predicted output estimate: 9.652754108607597


/usr/local/lib/python3.12/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 5 of parameter k1__k2__length_scale is close to the specified upper bound 1000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


###Week 11

Function: 1

Suggested next input (x_next): 0.328055-0.568992

Predicted output estimate: -1.5010868780465246e-05

Function: 2

Suggested next input (x_next): 0.690308-0.961842

Predicted output estimate: 0.625917696589368

Function: 3

Suggested next input (x_next): 0.510739-0.497528-0.443501

Predicted output estimate: -0.021569856598383508

Function: 4

Suggested next input (x_next): 0.390192-0.428036-0.33289-0.405141

Predicted output estimate: -0.7662211030218735

Function: 5

Suggested next input (x_next): 0.16303-1.0-1.0-1.0

Predicted output estimate: 4242.028572332445

Function: 6

Suggested next input (x_next): 0.463007-0.321444-0.537318-0.70508-0.0

Predicted output estimate: -0.2165142492871298

Function: 7

Suggested next input (x_next): 0.0-0.244433-0.41669-0.288534-0.326959-0.680382

Predicted output estimate: 2.296553357429581

Function: 8

Suggested next input (x_next): 0.090796-0.13846-0.178644-0.020794-0.996394-0.533622-0.231853-0.939059

Predicted output estimate: 9.652754108607597

###Week 10

Function: 1

Suggested next input (x_next): 0.329036-0.431477

Predicted output estimate: -2.6398105065961967e-05

Function: 2

Suggested next input (x_next): 0.684762-0.956283

Predicted output estimate: 0.5979956515253373

Function: 3

Suggested next input (x_next): 0.541094-0.476141-0.429833

Predicted output estimate: -0.014016835391371288

Function: 4

Suggested next input (x_next): 0.379054-0.415621-0.376105-0.4053

Predicted output estimate: -1.3280666630432822

Function: 5

Suggested next input (x_next): 0.161038-1.0-1.0-1.0

Predicted output estimate: 4234.224157433494

Function: 6

Suggested next input (x_next): 0.485318-0.35441-0.532001-0.735756-0.002593

Predicted output estimate: -0.5047795229646672

Function: 7

Suggested next input (x_next): 0.0-0.246499-0.426534-0.258151-0.365936-0.706056

Predicted output estimate: 2.185576374134149

Function: 8

Suggested next input (x_next): 0.128787-0.124725-0.195336-0.024228-1.0-0.534909-0.22382-0.947785

Predicted output estimate: 9.731860346327224


###Week 9

Function: 1

Suggested next input (x_next): 0.20084-0.740525

Predicted output estimate: -2.2611736184759493e-05

Function: 2

Suggested next input (x_next): 0.697663-0.999431

Predicted output estimate: 0.6019257317340984

Function: 3

Suggested next input (x_next): 0.5116-0.466301-0.440801

Predicted output estimate: -0.0073684692582915295

Function: 4

Suggested next input (x_next): 0.337602-0.314128-0.323601-0.447251

Predicted output estimate: 0.2144506112489193

Function: 5

Suggested next input (x_next): 0.106033-1.0-1.0-1.0

Predicted output estimate: 4393.815187907966

Function: 6

Suggested next input (x_next): 0.463141-0.457153-0.632226-0.697789-0.0

Predicted output estimate: -0.36433352926084206

Function: 7

Suggested next input (x_next): 0.0-0.273026-0.493981-0.248861-0.418811-0.645849

Predicted output estimate: 2.3946101918432645

Function: 8

Suggested next input (x_next): 0.032703-0.075576-0.154423-0.035445-0.99059-0.497687-0.174691-0.986528

Predicted output estimate: 9.73540021240301


### Week 8
Function: 1

Suggested next input (x_next): 0.354466-0.463725

Predicted output estimate: -6.573937079029337e-05

Function: 2

Suggested next input (x_next): 0.684793-0.736977

Predicted output estimate: 0.5478891261102885

Function: 3

Suggested next input (x_next): 0.671012-0.54088-0.410996

Predicted output estimate: -0.07154796838749342

Function: 4

Suggested next input (x_next): 0.462778-0.459808-0.427924-0.383695

Predicted output estimate: 1.4924733828472245

Function: 5

Suggested next input (x_next): 0.450158-0.310399-0.558612-0.719998

Predicted output estimate: 416.6177720858795

Function: 6

Suggested next input (x_next): 0.352618-0.580837-0.570192-0.621771-0.372965

Predicted output estimate: -0.5151560075399452

Function: 7

Suggested next input (x_next): 0.391166-0.399851-0.48006-0.389445-0.406096-0.616639

Predicted output estimate: 1.7499281039217427

Function: 8

Suggested next input (x_next): 0.416048-0.59922-0.417048-0.424239-0.551884-0.585992-0.386618-0.514817

Predicted output estimate: 9.054074978896033



# **## Week7**
Function: 1

Suggested next input (x_next): 0.961902-0.54151

Predicted output estimate: 0.0003957174409107497

Function: 2

Suggested next input (x_next): 0.686326-0.048467

Predicted output estimate: 0.5931130065069816

Function: 3

Suggested next input (x_next): 0.785819-0.951069-0.007398

Predicted output estimate: -0.07821291139128447

Function: 4

Suggested next input (x_next): 0.408168-0.42467-0.350333-0.464759

Predicted output estimate: 0.3892309433321408

Function: 5

Suggested next input (x_next): 0.165658-0.993001-0.985802-0.828885

Predicted output estimate: 3679.3006173709423

Function: 6

Suggested next input (x_next): 0.399356-0.472775-0.272075-0.930991-0.004327

Predicted output estimate: -0.33820016205812065

Function: 7

Suggested next input (x_next): 0.051607-0.242133-0.329607-0.131528-0.404377-0.688463

Predicted output estimate: 1.993543925493233

Function: 8

Suggested next input (x_next): 0.065492-0.074877-0.151838-0.036631-0.882332-0.83748-0.404222-0.901998

Predicted output estimate: 9.877420081086175


# **## Week6**

Function: 1

Suggested next input (x_next): 0.147927-0.999351

Predicted output estimate: 0.004738142161169105

Function: 2

Suggested next input (x_next): 0.576225-0.995582

Predicted output estimate: 1.0778257373391364

Function: 3

Suggested next input (x_next): 0.306294-0.000901-0.737840

Predicted output estimate: 0.03597608387388612

Function: 4

Suggested next input (x_next): 0.362616-0.390935-0.358916-0.408803

Predicted output estimate: 0.4543251119036995

Function: 5

Suggested next input (x_next): 0.379668-0.980190-0.997527-0.939278

Predicted output estimate: 4366.357278991082

Function: 6

Suggested next input (x_next): 0.377302-0.427069-0.554688-0.835249-0.037887

Predicted output estimate: -0.23066364279575358

Function: 7

Suggested next input (x_next): 0.002100-0.111359-0.320991-0.376242-0.229457-0.955090

Predicted output estimate: 2.464740596873976

Function: 8

Suggested next input (x_next): 0.026980-0.613170-0.025877-0.359845-0.896836-0.008212-0.028519-0.131032

Predicted output estimate: 10.435474844266569

